In [ ]:
!pip install coremltools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.2 MB/s eta 0:00:00


In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()

!rm -rf ~/.kaggle

!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d l3llff/flowers

 92% 209M/228M [00:02<00:00, 122MB/s]
100% 228M/228M [00:02<00:00, 112MB/s]


In [ ]:
!unzip -q flowers.zip

In [ ]:
!pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders

splitfolders.ratio(
    'flowers',
    './content',
    ratio=(0.7, 0.2, 0.1)
)

Copying files: 15740 files [00:03, 4934.34 files/s]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.4,
    rotation_range=0.4,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_data = train_datagen.flow_from_directory('train',
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               shuffle=True)

test_data = train_datagen.flow_from_directory('test',
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               shuffle=False)

val_data = train_datagen.flow_from_directory('val',
                                               target_size=(256, 256),
                                               class_mode='categorical',
                                               shuffle=False)

Found 11011 images belonging to 16 classes.
Found 1588 images belonging to 16 classes.
Found 3141 images belonging to 16 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
base_model = InceptionV3(include_top=False,
                         weights='imagenet',
                         input_shape=(256, 256, 3),
                         classes=16)

base_model.trainable = False

inputs = tf.Input((256, 256, 3))
x = base_model(inputs, training=False)
x = tf.layers.GlobalAveragePooling2D()(x)
output = tf.layers.Dense(16, 'softmax')(x)

model = tf.models.Model(inputs, output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 21,835,568
Trainable params: 32,784
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.metrics.CategoricalAccuracy()])

model.fit(train_data,
          epochs=10,
          validation_data=(test_data))

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


345/345 [==============================] - 186s 500ms/step - loss: 1.0525 - categorical_accuracy: 0.6906 - val_loss: 0.7365 - val_categorical_accuracy: 0.7853
Epoch 2/10
345/345 [==============================] - 169s 491ms/step - loss: 0.6028 - categorical_accuracy: 0.8156 - val_loss: 0.6370 - val_categorical_accuracy: 0.8092
Epoch 3/10
345/345 [==============================] - 168s 488ms/step - loss: 0.5023 - categorical_accuracy: 0.8452 - val_loss: 0.5828 - val_categorical_accuracy: 0.8262
Epoch 4/10
345/345 [==============================] - 170s 492ms/step - loss: 0.4680 - categorical_accuracy: 0.8554 - val_loss: 0.5356 - val_categorical_accuracy: 0.8419
Epoch 5/10
345/345 [==============================] - 170s 493ms/step - loss: 0.4274 - categorical_accuracy: 0.8634 - val_loss: 0.5055 - val_categorical_accuracy: 0.8526
Epoch 6/10
345/345 [==============================] - 171s 495ms/step - loss: 0.3938 - categorical_accuracy: 0.8719 - val_loss: 0.5227 - val_categorical_accuracy

In [ ]:
import coremltools as ct

cml_model = ct.convert(model,
                     inputs=[ct.ImageType(color_layout="RGB", scale=1/127.0, bias=[-1, -1, -1], )],
                     classifier_config=ct.ClassifierConfig(['astilbe', 'bellflower', 
                                                            'black_eyes_susan',
                                                            'calendula',
                                                            'california_poppy',
                                                            'carnation',
                                                            'common_daisy',
                                                            'coreopsis',
                                                            'daffodil',
                                                            'dandelion',
                                                            'iris',
                                                            'magnolia',
                                                            'rose',
                                                            'sunflower',
                                                            'tulip',
                                                            'water_lily'])
                    )

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 976/976 [00:02<00:00, 394.25 ops/s]


In [ ]:
cml_model

input {
  name: "input_4"
  type {
    imageType {
      width: 256
      height: 256
      colorSpace: RGB
      imageSizeRange {
        widthRange {
          lowerBound: 256
          upperBound: 256
        }
        heightRange {
          lowerBound: 256
          upperBound: 256
        }
      }
    }
  }
}
output {
  name: "Identity"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "Identity"
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.12.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "6.3.0"
  }
}

In [ ]:
cml_model.save('FlowerClassifier.mlmodel')